In [1]:
import mlflow
import pandas as pd
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
import mlflow.sklearn
from mlflow.models.signature import infer_signature
import os

In [11]:
mlflow.set_tracking_uri('http://localhost:5000')

os.environ['AWS_ACCESS_KEY_ID'] = 'minio_user'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'minio_password'
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'
endpoint = 'http://localhost:9000'
os.environ['MLFLOW_S3_ENDPOINT_URL'] = endpoint

In [25]:
model_name = "my-registered-model"

iris = datasets.load_iris()
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names) 

iris_train = iris_df.iloc[:-1]
iris_test = iris_df.iloc[-1:]

experiment_id = mlflow.set_experiment('model-demo')
with mlflow.start_run():
    clf = RandomForestClassifier(max_depth=7, random_state=0)
    clf.fit(iris_train, iris.target[:-1])
    signature = infer_signature(iris_train, clf.predict(iris_train))
    mlflow.sklearn.log_model(clf, "iris_rf", signature=signature, registered_model_name=model_name)

Registered model 'my-registered-model' already exists. Creating a new version of this model...
2025/02/23 17:44:48 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: my-registered-model, version 5


🏃 View run wise-croc-83 at: http://localhost:5000/#/experiments/1/runs/de398ca635cf475ca056e28b1077e7ec
🧪 View experiment at: http://localhost:5000/#/experiments/1


Created version '5' of model 'my-registered-model'.


In [26]:
signature

inputs: 
  ['sepal length (cm)': double (required), 'sepal width (cm)': double (required), 'petal length (cm)': double (required), 'petal width (cm)': double (required)]
outputs: 
  [Tensor('int64', (-1,))]
params: 
  None

In [28]:
import mlflow.pyfunc

model_version = 1

model = mlflow.pyfunc.load_model(
    model_uri=f"models:/{model_name}/{model_version}"
)

result = model.predict(iris_test)

print(result)

[2]


In [29]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

stage = 'Staging'


client.set_registered_model_alias(name=model_name, alias=stage, version=str(model_version))

model = mlflow.pyfunc.load_model(
    model_uri=f"models:/{model_name}@{stage}"
)

result = model.predict(iris_test)

print(result)

[2]


In [30]:
from pprint import pprint

for rm in client.search_registered_models():
    pprint(dict(rm), indent=4)

{   'aliases': {'champ': '2'},
    'creation_timestamp': 1740089133875,
    'description': '',
    'last_updated_timestamp': 1740324494952,
    'latest_versions': [   <ModelVersion: aliases=[], creation_timestamp=1740324494952, current_stage='None', description='', last_updated_timestamp=1740324494952, name='Impersonator', run_id='95da7fabe08c42339044fc3fc74e6cbc', run_link='', source='s3://bucket/3/95da7fabe08c42339044fc3fc74e6cbc/artifacts/langchain_model', status='READY', status_message=None, tags={}, user_id='', version='2'>],
    'name': 'Impersonator',
    'tags': {}}
{   'aliases': {},
    'creation_timestamp': 1740089080295,
    'description': '',
    'last_updated_timestamp': 1740089080329,
    'latest_versions': [   <ModelVersion: aliases=[], creation_timestamp=1740089080329, current_stage='None', description='', last_updated_timestamp=1740089080329, name='Inpersonator', run_id='efb9d799e50945f1982d6ac1e3dc337b', run_link='', source='s3://bucket/3/efb9d799e50945f1982d6ac1e3dc

In [31]:
client.delete_registered_model(name=model_name)

In [32]:
for rm in client.search_registered_models():
    pprint(dict(rm), indent=4)

{   'aliases': {'champ': '2'},
    'creation_timestamp': 1740089133875,
    'description': '',
    'last_updated_timestamp': 1740324494952,
    'latest_versions': [   <ModelVersion: aliases=[], creation_timestamp=1740324494952, current_stage='None', description='', last_updated_timestamp=1740324494952, name='Impersonator', run_id='95da7fabe08c42339044fc3fc74e6cbc', run_link='', source='s3://bucket/3/95da7fabe08c42339044fc3fc74e6cbc/artifacts/langchain_model', status='READY', status_message=None, tags={}, user_id='', version='2'>],
    'name': 'Impersonator',
    'tags': {}}
{   'aliases': {},
    'creation_timestamp': 1740089080295,
    'description': '',
    'last_updated_timestamp': 1740089080329,
    'latest_versions': [   <ModelVersion: aliases=[], creation_timestamp=1740089080329, current_stage='None', description='', last_updated_timestamp=1740089080329, name='Inpersonator', run_id='efb9d799e50945f1982d6ac1e3dc337b', run_link='', source='s3://bucket/3/efb9d799e50945f1982d6ac1e3dc